In [ ]:
# Data directory
data_dir = './data'
!ls './data'

01


In [ ]:
!pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 36.6 MB/s eta 0:00:00


In [ ]:
# !pip install sklearn_crfsuite
# sklearn_crfsuite
!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to /tmp/pip-install-m4v4rnxb/sklearn-crfsuite_243b1338484447348da365e90ef95a8b
  Running command git clone --filter=blob:none --quiet https://github.com/MeMartijn/updated-sklearn-crfsuite.git /tmp/pip-install-m4v4rnxb/sklearn-crfsuite_243b1338484447348da365e90ef95a8b
  Resolved https://github.com/MeMartijn/updated-sklearn-crfsuite.git to commit 675038761b4405f04691a83339d04903790e2b95
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00
  Created wheel for sklearn_crfsuite: filename=sklearn_crfsuite-0.3.6-py2.py3-none-any.whl size=10889 sha256=18c8c5c88ba4c6718d5ab75681d3e950032568ab45c802ea35ca272e755f7526
  Stored in directory: /tmp/pip-ephem-wheel-cache-xlix7akl/wheels/fb/c8/8a/95b4eccd3a273adbfb0a08d7f0e96d45d9f4aee82015d293c2
Successfully built sklear

In [ ]:
import pandas as pd 
import re
import os
from nltk.tokenize import RegexpTokenizer
#from pythainlp.tokenize import word_tokenize
# from pythainlp.tag import pos_tag

In [ ]:
def read_raw_text(filename):
  with open(filename, 'r', encoding = 'utf-8') as file:
    document = file.read()
  return document
  
def read_ann_file(PATH, filename):  #filename e.g. 01_nut.a/xxaa.ann

  PATH = PATH
  document = read_raw_text(PATH + filename[:-4] + '.txt')
  df = pd.read_csv(PATH + filename, sep='^([^\s]*)\s', engine='python', header=None).drop(0, axis=1)
    
    
  token_df = df[df[1].str.contains('T')]
  print(token_df)
  list_tokens = []
    
  seek = 0
    
  for index, row in token_df.iterrows():
    
    text = re.findall('\t.*', row[2])[0][1:]
    print(text)
    entityLabel, start, end = re.findall('.*\t', row[2])[0][:-1].split(' ')
    start, end = int(start), int(end)
        
    if seek == start:
      res = [document[start:end], start, end, entityLabel]
      list_tokens.append(res)
            
    else:

      #print(seek, start)
      res = [document[seek:start], seek, start, 'O']
      list_tokens.append(res)
            
      res = [document[start:end], start, end, entityLabel]
      list_tokens.append(res)
            
    seek = end
    
    
  result_text = ''
  print(list_tokens)
  for t, start, end, ent in list_tokens:
    text = f'[{ent}]{t}[/{ent}]'
    result_text += text
        
    
  return result_text , list_tokens

def read_all_file(PATH):

  PATH = PATH 
  assignee_folder_list = os.listdir(PATH)
  
  result = {'text' : []} 
  for assignee_folder in assignee_folder_list:
    text_folder_list = sorted(os.listdir(PATH + assignee_folder)) 
    text_folder_list = [i for i in text_folder_list if i[-3:] in ['ann', 'txt']]
    text_folder_list = set(map(lambda x : x[:-4], text_folder_list))
    
    for text_folder in text_folder_list:

      filename = assignee_folder + '/' + text_folder + '.ann'
      text = read_ann_file(PATH, filename)
      result['text'].append(text)
    
  df = pd.DataFrame(result)
  return df

# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
  text=text.strip()
  text=re.sub("<[^>]*>","",text)
  text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')#  ตัดการกับพวกไม่มี tag word
  text=re.sub("(\[\w+\])","***\\1",text)
  text2=[]
  for i in text.split('***'):
    if "[" in i:
      text2.append(i)
    else:
      text2.append("[word]"+i+"[/word]")
  text="".join(text2)#re.sub("[word][/word]","","".join(text2))
  return text.replace("[word][/word]","")
  
# เตรียมตัวตัด tag ด้วย re
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น 

def text2conll2002(text, pos=True):
  text=toolner_to_tag(text)
  text=text.replace("''",'"')
  text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
  tag=tokenizer.tokenize(text) # แยก tag ออกมาจากข้อความ
  j=0
  conll2002="" # ประกาศตัวแปรเก็บ conll2002
  for tagopen,text,tagclose in tag:
    word_cut = word_tokenize(text, engine="newmm")
    i=0 
    txt5=""
    while i<len(word_cut):
      if word_cut[i]=="''" or word_cut[i]=='"':pass
      elif i==0 and tagopen!='word': # ไม่เป็น tag [word] และเป็น i หรือตัวเริ่มต้น tag
        txt5+=word_cut[i]
        txt5+='\t'+'B-'+tagopen
      elif tagopen!='word':
        txt5+=word_cut[i]
        txt5+='\t'+'I-'+tagopen
      else:
        txt5+=word_cut[i]
        txt5+='\t'+'O'
      txt5+='\n'
      i+=1
      conll2002+=txt5
  if pos==False:
    return conll2002
  return postag(conll2002)

# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
# print(text2conll2002(t,pos=False))
def postag(text):
  listtxt=[i for i in text.split('\n') if i!='']
  list_word=[]
  for data in listtxt:
    list_word.append(data.split('\t')[0])
  #print(text)
  list_word=pos_tag(list_word,engine="perceptron")
  text=""
  i=0
  for data in listtxt:
    text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\n'
    i+=1
  return text

# เอา dataframe มาเเยกออกเป็น list
def get_data(df):
  data = []
  for index, row in df.iterrows():
    data.append(row['text']) 
  return data

def alldata(lists):
  text = ""
  for data in lists:
    text+=text2conll2002(data)
    text+='\n'
  return text 

def alldata_list(lists):
    data_all=[]
    for data in lists:
        data_num=[]
        try:
            txt=text2conll2002(data,pos=True).split('\n') # นำไปแปลงเป็น conll2002
            for d in txt:
                tt=d.split('\t')
                if d!="":
                    if len(tt)==3:
                        data_num.append((tt[0],tt[1],tt[2]))
                    else:
                        data_num.append((tt[0],tt[1]))
            #print(data_num)
            data_all.append(data_num)
        except:
            print(data)
    #print(data_all)
    return data_all


def alldata_list_str(lists):
	string=""
	for data in lists:
		string1=""
		for j in data:
			string1+=j[0]+"	"+j[1]+"	"+j[2]+"\n"
		string1+="\n"
		string+=string1
	return string

def get_data_tag(listd):
	list_all=[]
	c=[]
	for i in listd:
		if i !='':
			c.append((i.split("\t")[0],i.split("\t")[1],i.split("\t")[2]))
		else:
			list_all.append(c)
			c=[]
	return list_all
def getall(lista):
    ll=[]
    for i in lista:
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(i)
    return ll

In [ ]:
# read_ann_file(data_dir, '01/xaaa.ann')

In [ ]:
df = read_all_file(data_dir)

In [ ]:
data1=getall(get_data(df))
datatofile=alldata_list(data1)
tt=[]
with open("data"+"-pos.conll","w") as f:
    i=0
    while i<len(datatofile):
        for j in datatofile[i]:
            f.write(j[0]+"\t"+j[1]+"\t"+j[2]+"\n")
        if i+1<len(datatofile):
            f.write("\n")
        i+=1

with open("data"+".conll","w") as f:
    i=0
    while i<len(datatofile):
        for j in datatofile[i]:
            f.write(j[0]+"\t"+j[2]+"\n")
        if i+1<len(datatofile):
            f.write("\n")
        i+=1

In [ ]:
len(datatofile)

712

In [ ]:
from sklearn_crfsuite import scorers,metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate,train_test_split
import sklearn_crfsuite
def doc2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    # Features from current word
    features={
        'word.word': word,
        'word.isspace':word.isspace(),
        'postag':postag,
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        prevword = doc[i-1][0]
        postag1 = doc[i-1][1]
        features['word.prevword'] = prevword
        features['word.previsspace']=prevword.isspace()
        features['word.prepostag'] = postag1
        features['word.prevwordisdigit'] = prevword.isdigit()
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        postag1 = doc[i+1][1]
        features['word.nextword'] = nextword
        features['word.nextisspace']=nextword.isspace()
        features['word.nextpostag'] = postag1
        features['word.nextwordisdigit'] = nextword.isdigit()
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features

def extract_features(doc):
    return [doc2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [tag for (token,postag,tag) in doc]


X_data = [extract_features(doc) for doc in datatofile] # เอา คำ แยกออกมา
y_data = [get_labels(doc) for doc in datatofile] # เอา tag แยกออกมา

X, X_test, y, y_test = train_test_split(X_data, y_data, test_size=0.1) # แบ่ง 0.1 หรือ 10%
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=500,
    all_possible_transitions=True,
    model_filename="data"+"-pos.model0" # ตั้งชื่อโมเดล
)
crf.fit(X, y); # train

labels = list(crf.classes_)
print(labels)
labels.remove('B-O')
labels.remove('I-O')
y_pred = crf.predict(X_test)
e=metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)
print(e) # โชว์ประสิทธิภาพ
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

['B-O', 'I-O', 'B-PERSON', 'I-PERSON', 'B-VEHICLE', 'B-WEAPON', 'B-DATE', 'I-DATE', 'B-TIME', 'I-TIME', 'B-ORGANIZATION', 'I-ORGANIZATION', 'B-LOCATION', 'I-LOCATION', 'I-VEHICLE', 'B-COLOR', 'I-COLOR', 'B-LP', 'I-LP', 'I-WEAPON', 'B-OBJECT', 'I-OBJECT']
0.6983039836480242
                precision    recall  f1-score   support

       B-COLOR      0.408     0.635     0.497        63
       I-COLOR      0.308     0.596     0.406        47
        B-DATE      0.976     0.941     0.958       340
        I-DATE      0.983     0.952     0.967       475
    B-LOCATION      0.876     0.799     0.836      1038
    I-LOCATION      0.880     0.694     0.776      1310
          B-LP      0.615     0.508     0.557        63
          I-LP      0.531     0.589     0.559       129
      B-OBJECT      0.676     0.481     0.562       104
      I-OBJECT      0.552     0.577     0.564       137
B-ORGANIZATION      0.728     0.725     0.726       498
I-ORGANIZATION      0.568     0.599     0.583       7

In [ ]:
def get_ner(text):
    word_cut=word_tokenize(text,engine="newmm")
    list_word=pos_tag(word_cut,engine='perceptron')
    X_test = extract_features([(data,list_word[i][1]) for i,data in enumerate(word_cut)])
    y_=crf.predict_single(X_test)
    return [(word_cut[i],list_word[i][1],data) for i,data in enumerate(y_)]


while True:
    t=input("Text : ")
    print(get_ner(t))

Text : ปิดเมืองล่า 2 มือปืนยิงหัววินดับ กลางเมืองพัทยา ผบ.ตร.จี้ลากคอ ตำรวจตั้งข้อหา 2 มือปืนจ่อยิงเบ้าตาวิน จยย.ดับกลางเมืองพัทยา ผบ.ตร.สั่งไล่ล่า-กดดันลากคอดำเนินคดี เน้นย้ำระมัดระวัง เหตุคนร้ายมีปืน ชี้หากขัดขืนเหิมต่อสู้ ไฟเขียวขั้นเด็ดขาดตามหลักยุทธวิธี&nbsp;พร้อมให้กำลังใจ ตร.ทำงาน สร้างความเชื่อมั่นให้ประชาชน&nbsp; &nbsp;&nbsp;เมื่อวันที่ 29 ก.ค.63 พ.ต.อ.กฤษณะ พัฒนเจริญ รองโฆษก ตร. เปิดเผยถึงความคืบหน้า กรณีที่สื่อนำเสนอข่าว &quot;ฆ่าอุกอาจกลางเมืองพัทยา มือปืนจ่อยิงเบ้าตาวินจักรยานยนต์&quot; ว่า ได้รับรายงานจาก สภ.เมืองพัทยา ว่าเมื่อเวล... ตำรวจตั้งข้อหา 2 มือปืนจ่อยิงเบ้าตาวิน จยย.ดับกลางเมืองพัทยา ผบ.ตร.สั่งไล่ล่า-กดดันลากคอดำเนินคดี เน้นย้ำระมัดระวัง เหตุคนร้ายมีปืน ชี้หากขัดขืนเหิมต่อสู้ ไฟเขียวขั้นเด็ดขาดตามหลักยุทธวิธี&nbsp;พร้อมให้กำลังใจ ตร.ทำงาน สร้างความเชื่อมั่นให้ประชาชน&nbsp; &nbsp;&nbsp;เมื่อวันที่ 29 ก.ค.63 พ.ต.อ.กฤษณะ พัฒนเจริญ รองโฆษก ตร. เปิดเผยถึงความคืบหน้า กรณีที่สื่อนำเสนอข่าว &quot;ฆ่าอุกอาจกลางเมืองพัทยา มือปืนจ่อยิงเบ้าตาวินจักรยานยนต์&quot; ว่า ได้รับร

KeyboardInterrupt: ignored